In [1]:
!pip install tflearn
!pip install tensorflow==1.15

     |████████████████████████████████| 102kB 4.3MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=6c8cbb66e0b5056d472e5960d5a2af0c1a9f39f8da82d083fd263f3639b4be15
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn
     |████████████████████████████████| 412.3MB 41kB/s 
     |████████████████████████████████| 3.8MB 33.9MB/s 
     |████████████████████████████████| 512kB 39.6MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=76488fc67040dc72c60d96afffaa4542f9df00b32f0598b43cbe8627675f7def
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Succe

In [2]:
import os
from six import moves
import ssl
import tflearn
from tflearn.data_utils import *

In [3]:
path = "US_Cities.txt"
if not os.path.isfile(path):
    context = ssl._create_unverified_context()
    moves.urllib.request.urlretrieve("https://raw.githubusercontent.com/tflearn/tflearn.github.io/master/resources/US_Cities.txt", path)

In [4]:
maxlen = 20

string_utf8 = open(path, "r").read()
X, Y, char_idx =     string_to_semi_redundant_sequences(string_utf8, seq_maxlen=maxlen, redun_step=3)

Vectorizing text...
Text total length: 207,041
Distinct chars   : 61
Total sequences  : 69,007


In [5]:
g = tflearn.input_data(shape=[None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)



Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead





In [6]:
m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=5.0,
                              checkpoint_path='model_us_cities')



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where







In [7]:
for i in range(40):
    seed = random_sequence_from_string(string_utf8, maxlen)
    m.fit(X, Y, validation_set=0.1, batch_size=128,
          n_epoch=1, run_id='us_cities')
    print("-- TESTING...")
    print("-- Test with temperature of 1.2 --")
    print(m.generate(30, temperature=1.2, seq_seed=seed).encode('utf-8'))
    print("-- Test with temperature of 1.0 --")
    print(m.generate(30, temperature=1.0, seq_seed=seed).encode('utf-8'))
    print("-- Test with temperature of 0.5 --")
    print(m.generate(30, temperature=0.5, seq_seed=seed).encode('utf-8'))

Training Step: 9134  | total loss: 0.58206 | time: 24.321s
| Adam | epoch: 019 | loss: 0.58206 -- iter: 49408/62106


KeyboardInterrupt: ignored